# แบบฝึกหัด: การตรวจจับการฉ้อโกงบัตรเครดิต

พิจารณาชุดข้อมูล [Credit card frauds](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud) จาก Kaggle ซึ่งสามารถดาวน์โหลดได้ดังนี้



In [1]:
import gdown
file_id = '1pqOkV39Sk6_D64sVfkMXXzRLwe0y9rFE'
output = 'creditcard.zip'
gdown.download(f"https://drive.google.com/uc?export=download&confirm=pbef&id={file_id}", output)

Downloading...
From: https://drive.google.com/uc?export=download&confirm=pbef&id=1pqOkV39Sk6_D64sVfkMXXzRLwe0y9rFE
To: /content/creditcard.zip
100%|██████████| 69.2M/69.2M [00:02<00:00, 30.4MB/s]


'creditcard.zip'

In [2]:
# แตกไฟล์ zip
!unzip -o creditcard.zip

Archive:  creditcard.zip
  inflating: creditcard.csv          


โดยข้อมูลที่ให้ เป็นข้อมูลธุรกรรมบัตรเครดิต 2 วัน ในเดือนกันยายน 2013 จากธนาคารแห่งหนึ่งในยุโรป โดยมีรายละเอียดข้อมูลดังนี้

- **ตัวแปรต้น**
  - คอลัมน์ที่ถูกตั้งชื่อเป็น V1, V2, …, และ V28 ซึ่งเป็นค่าที่ถูกแปลงหรือลดมิติข้อมูลมาแล้ว ด้วยเทคนิค [Principal Component Analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)  ซึ่งสาเหตุของการไม่เผยแพร่ข้อมูลต้นฉบับ เนื่องมาจากเป็นข้อมูลส่วนบุคคลของลูกค้า
  - คอลัมน์ Time กับ Amount ที่เป็นข้อมูลต้นฉบับที่ไม่ผ่าน PCA
- **ตัวแปรตาม** เป็นไปได้ 2 ค่าคือ
  - Class 0: ธุรกรรมปกติ (no fraud)
  - Class 1: ธุรกรรมผิดปกติ (frauds)
  

จากข้อมูลข้างต้น จงทำนายความผิดปกติของธุรกรรม โดยใช้โมเดลต่อไปนี้
  1. Logistic regression (`LogisticRegression`)
  2. Decision tree (`DecisionTreeClassifier`)
  3. Random forest (`RandomForestClassifier`, [คู่มือ](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html))

โดยให้ แบ่งข้อมูล **train 70% และ test 30%** ซึ่งต้องไม่ลืมการ Standardize ข้อมูลด้วย และในส่วนการเทรนโมเดล อาจใช้ `GridSearchCV` มาช่วยด้วยก็ได้

จากโมเดลที่ได้
1. ให้รายงานประสิทธิภาพของโมเดลโดยใช้ Confusion matrix
2. ให้ระบุว่าโมเดลใดดีที่สุด
3. ลองวิเคราะห์ว่า ทำไมการรายงานและเปรียบเทียบประสิทธิภาพของโมเดลโดยใช้ Accuracy (`accuracy_score`) จึงไม่ใช่สิ่งที่ถูกต้อง

ทั้งนี้ นักศึกษายังไม่เคยเรียนว่าโมเดล Random forest คืออะไร โดยคร่าว ๆ แล้ว มันคือโมเดลตระกูลต้นไม้ ที่จะสร้าง Decision tree หลาย ๆ ต้นขึ้นมา โดยในการตัดสินใจ จะพิจารณานำผลการทำนายจาก Decision tree แต่ละต้น มาโหวตหรือเฉลี่ย เพื่อฟันธงผลลัพธ์สุดท้าย เราเรียกเทคนิคการใช้โมเดลหลาย ๆ ตัว มาช่วยกันตัดสินใจว่าโมเดลแบบ **Ensemble** 

> การที่ Random forest สร้างต้นไม้หลายต้นจากข้อมูลเดียวกัน หากต้องสอนต้นไม้ทุกต้นด้วยข้อมูลที่มีลำดับเหมือนกันหมด ก็จะกลายเป็นว่าได้ต้นไม้ที่เหมือน ๆ กัน ดังนั้น ในทางปฏิบัติ Random forest ก่อนที่จะสร้างต้นไม้แต่ละต้น Random forest จะทำการ shuffle ข้อมูลทั้งหมดก่อน จึงเป็นที่มาของคำว่า Random ในชื่อของ Random forest

สำหรับรูปแบบการใช้ `RandomForestClassifier` จะเหมือนกับ `DecisionTreeClassifier` โดยที่จะต้องกำหนดว่า ต้องการสร้างต้นไม้กี่ต้น (`n_estimators`)

```python
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=จำนวนต้นไม้,
     max_depth=จำนวนชั้นของต้นไม้ในการตัดสินใจ,   # เหมือนของ DecisionTreeClassifier
     min_samples_split=จำนวนจุดข้อมูลขั้นต่ำที่จะทำการแตกกิ่ง, # เหมือนของ DecisionTreeClassifier
     min_samples_leaf=จำนวนจุดข้อมูลขั้นต่ำที่ปลายกิ่งต้องมี,    # เหมือนของ DecisionTreeClassifier
     random_state=ค่าซีดล็อคผลการสุ่ม)        # เหมือนของ DecisionTreeClassifier
```

ซึ่งศึกษาวิธีการใช้เพิ่มเติมได้จาก[คู่มือ](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)